# CIIC5015: Intro to Artificial Intelligence
### Example 4: Intro to PyTorch - Fully Connected Neural Netowrks
#### Import the necessary libraries
- PyTorch
- TorchVision
- Numpy
- Matplotlib

In [ ]:
import torch
import torchvision
import torchinfo
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

Set up data loaders (read data from files or urls)

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

batch_size = 4

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog','horse', 'ship', 'truck')

Use Matplotlib to see a few data examples

In [ ]:
def imgshow(img):
    img = img / 2 + 0.5 # Unormalize the data
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()
    
# Fetch some random training images
data_iterator = iter(train_loader)
images, labels = next(data_iterator) # Return a batch of images with their labels
print(f"images.shape: {images.shape}")
random_image = images[0]
print(f"random_image.shape: {random_image.shape}")
print(f"random_image.view(-1).shape: {random_image.view(-1).shape}")
    
# Show random images
imgshow(torchvision.utils.make_grid(images))
print(' '.join(f"{classes[labels[j]]:5s}" for j in range(batch_size)))
    


Let's understand the shape of the data.

First let's take a look at the image structures

In [ ]:
print(f"images.shape: {images.shape}")

`images` is a batch of 4 images. Each image has a shape of 3 color channels x 32 x 32 pixels

In [ ]:
image = images[0]
print(f"image: {image.shape}")

In [ ]:
flattened_image = image.view(-1)
print(flattened_image)
print(f"flattened_image.shape: {flattened_image.shape}")

In [ ]:
"""
Take all images in batch and flatten each one to a 1D array of 3072 pixels

flattened_batch = tensor([
    [0.4588,  0.4745,  0.4667,  ..., -0.2157, -0.2314, -0.2471],
    [0.4588,  0.4742,  0.4667,  ..., -0.2157, -0.2314, -0.2471],
    [0.4588,  0.4345,  0.4667,  ..., -0.2157, -0.2314, -0.2471],
    [0.4588,  0.4785,  0.4667,  ..., -0.2157, -0.2314, -0.2471]
])

These will be 3072 features that we will feed to our neural network
"""

flattened_batch = images.view(-1, 3072)
print(f"flattened_batch.shape: {flattened_batch.shape}")


In [ ]:
class FCModel(torch.nn.Module):
    def __init__(self,input_size):
        super(FCModel, self).__init__()
        self.layer_1 = torch.nn.Linear(input_size, 30)
        self.layer_2 = torch.nn.Linear(30, 10)
        self.activation_1 = torch.nn.ReLU()
        self.out_activation = torch.nn.Softmax(dim=1)
    
    def forward(self, x):
        x = x.view(-1, 3072) # Flatten image taken from the batch
        x = self.layer_1(x) # Pass image through first layer
        x = self.activation_1(x) # Pass image through ReLU
        x = self.layer_2(x) # Pass image through second layer
        x = self.out_activation(x) # Pass image through softmax layer
        
        return x
    
neural_network = FCModel(3072) # Create model for images of 3072 pixels (number of features are number of pixels)


In [ ]:
batch_size = 64
torchinfo.summary(neural_network, input_size=(batch_size, 3, 32, 32), device='cpu', col_names=['input_size', 'output_size', 'num_params'])

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(neural_network.parameters(), lr=0.1)

In [ ]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs,labels = data
        
        optimizer.zero_grad()
        
        outputs = neural_network(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 2000 == 1999:
            print(f"[{epoch + 1}, {i + 1:5d}] loss: {(running_loss / 2000):3f}")
            running_loss = 0.0
            
print("Finished Training!")

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        
        # Calculate outputs by running images throuhg the NN
        outputs = neural_network(image)
        
        _, predicted = torch.max(outputs.data, 1)
        
        # Compute accuracy of NN
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print(f"Accuracy of the network on the 10000 test images: {100 * correct // total}%")